In [1]:
import metrics_helpers as indicators
import pickle as pk
import gc
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import traceback #needed to store full error tracebacks

import plotly

#import plotly.plotly as py
import plotly.tools as tls

In [2]:
import dash
from dash import dcc
from dash import html
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
#import dash_design_kit as ddk

In [3]:
import plotly.express as px



In [4]:
#build text list for use in functions and callbacks.
sent_vals = ['pattern', 'nltk', 'mod', 'subj']
plot_vals = ['ttest', 'boxplot', 'corr', 'reg']

#create each object in dicts.
pointless_div = html.Div(children=[html.H3(children='object not implemented')])
haz_graphs = {s:{p:pointless_div for p in plot_vals} for s in sent_vals}
prod_graphs = {s:{p:pointless_div for p in plot_vals} for s in sent_vals}

In [5]:
haz_graphs['pattern'].keys()

dict_keys(['ttest', 'boxplot', 'corr', 'reg'])

In [6]:
def get_table(df):#https://dash.plotly.com/datatable/conditional-formatting
    return dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            editable=True,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable="single",
            row_selectable="multi",
            row_deletable=True,
            selected_columns=[],
            selected_rows=[],
            page_action="native",
            page_current= 0,
            page_size= 10,
        )

def add_label(obj, lab):
    return html.Div([html.Label(lab, style={'flex-shrink':'2', 'flex-basis':'60%'}),obj])

In [7]:
#debug
import pandas as pd
from dash import dash_table

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')

fig = px.box(df, y="Number of Solar Plants")

In [8]:
#add graphs 
haz_graphs['pattern']['ttest'] = add_label(get_table(pat_haz_t), 'Pattern T-Test')
haz_graphs['pattern']['boxplot'] = dcc.Graph(figure=pat_haz_box)
haz_graphs['pattern']['reg'] = add_label(get_table(pat_haz_reg), 'Pattern Regression')

haz_graphs['nltk']['ttest'] = add_label(get_table(nltk_haz_t), 'NLTK T-Test')
haz_graphs['nltk']['boxplot'] = dcc.Graph(figure=nltk_haz_box)
haz_graphs['nltk']['reg'] = add_label(get_table(nltk_haz_reg), 'NLTK Regression')

haz_graphs['mod']['ttest'] = add_label(get_table(mod_haz_t), 'Modality T-Test')
haz_graphs['mod']['boxplot'] = dcc.Graph(figure=mod_haz_box)
haz_graphs['mod']['reg'] = add_label(get_table(mod_haz_reg), 'Modality Regression')

haz_graphs['subj']['ttest'] = add_label(get_table(subj_haz_t), 'Subjectivity T-Test')
haz_graphs['subj']['boxplot'] = dcc.Graph(figure=subj_haz_box)
haz_graphs['subj']['reg'] = add_label(get_table(subj_haz_reg), 'Subjectivity Regression')
##############################################
##############################################
prod_graphs['pattern']['ttest'] = add_label(get_table(pat_prod_t), 'Pattern T-Test')
prod_graphs['pattern']['boxplot'] = dcc.Graph(figure=pat_prod_box)
prod_graphs['pattern']['reg'] = add_label(get_table(pat_prod_reg), 'Pattern Regression')

prod_graphs['nltk']['ttest'] = add_label(get_table(nltk_prod_t), 'NLTK T-Test')
prod_graphs['nltk']['boxplot'] = dcc.Graph(figure=nltk_prod_box)
prod_graphs['nltk']['reg'] = add_label(get_table(nltk_prod_reg), 'NLTK Regression')

prod_graphs['mod']['ttest'] = add_label(get_table(mod_prod_t), 'Modality T-Test')
prod_graphs['mod']['boxplot'] = dcc.Graph(figure=mod_prod_box)
prod_graphs['mod']['reg'] = add_label(get_table(mod_prod_reg), 'Modality Regression')

prod_graphs['subj']['ttest'] = add_label(get_table(subj_prod_t), 'Subjectivity T-Test')
prod_graphs['subj']['boxplot'] = dcc.Graph(figure=subj_prod_box)
prod_graphs['subj']['reg'] = add_label(get_table(subj_prod_reg), 'Subjectivity Regression')


In [9]:


external_stylesheets = ['/Users/sma/Documents/INRAE internship/scrape-git/notebooks/assets/ugh.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options



app.layout = html.Div([
    dcc.Checklist(
        id='sent_checklist',
    options=[
        {'label': 'Pattern Sent', 'value': sent_vals[0]},
        {'label': 'NLTK Sent', 'value': sent_vals[1]},
        {'label': 'Modality', 'value': sent_vals[2]},
        {'label': 'Subjectivity', 'value': sent_vals[3]}
    ],
    value=['nltk'],
    labelStyle={'display': 'inline-block'}
),
    
    dcc.Checklist(
        id='prod_checklist',
    options=[
        {'label': 'T-Test', 'value': plot_vals[0]},
        {'label': 'BoxPlot', 'value': plot_vals[1]},
        {'label': 'Correlation', 'value': plot_vals[2]},
        {'label': 'Regression', 'value': plot_vals[3]}
    ],
    value=['boxplot','ttest'],
    labelStyle={'display': 'inline-block'}
),
    
    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='Hazards', value='tab-1'),
        dcc.Tab(label='Products', value='tab-2'),
        dcc.Tab(label='Explore', value='tab-3')
    ]),
    html.Div(id='tabs-example-content') #load in based on the callback function
])



#NOTE ON LAYOUT
#    * each ID must be used only ONCE __per tab__
@app.callback(Output('tabs-example-content', 'children'),
              Input('tabs-example', 'value'))
def render_content(tab):
    if tab == 'tab-1': #HAZARD
        return html.Div(children=[
            html.Div(id='haz-things')])
    
    
    elif tab == 'tab-2': #PRODUCT
                return html.Div(children=[
            html.Div(id='prod-things')])

    elif tab == 'tab-3': #EXPLORE
        return html.Div(children=[html.H1(children='''Not Implement :(''')])
        #TODO show corrs, show a graph where u choose variables, show counts?
        # corrs is most important.
        #https://plotly.com/python/heatmaps/

#structure: 8 callbacks. first check if the metric should be displayed, then add each plot to children
#then send to div.
###############ACTUAL GRAPH CONTENT DISPLAYS###############   
#HAZARDS TAB
@app.callback(Output('haz-things', 'children'),
              [Input('prod_checklist', 'value'), Input('sent_checklist', 'value')])
def render_content(values_in_plot_checklist, values_in_sent_checklist):
    c = []
    
    for v in sent_vals:
        if v in values_in_sent_checklist:
            # render the plots which are desired, in pattern section.
            
            if plot_vals[0] in values_in_plot_checklist:
                c.append(haz_graphs[v][plot_vals[0]])
            if plot_vals[1] in values_in_plot_checklist:
                c.append(haz_graphs[v][plot_vals[1]])
            if plot_vals[3] in values_in_plot_checklist:
                c.append(haz_graphs[v][plot_vals[3]])                  
    if c: #if not empty
        return html.Div(children=c, className='flex-container')#, style={'id':'container'}

#PRODUCTS TAB
@app.callback(Output('prod-things', 'children'),
              [Input('prod_checklist', 'value'), Input('sent_checklist', 'value')])
def render_content(values_in_plot_checklist, values_in_sent_checklist):
    c = []
    
    for v in sent_vals:
        if v in values_in_sent_checklist:
            # render the plots which are desired, in pattern section.
            
            if plot_vals[0] in values_in_plot_checklist:
                c.append(prod_graphs[v][plot_vals[0]])
            if plot_vals[1] in values_in_plot_checklist:
                c.append(prod_graphs[v][plot_vals[1]])
            if plot_vals[3] in values_in_plot_checklist:
                c.append(prod_graphs[v][plot_vals[3]])                  
    if c: #if not empty
        return html.Div(children=c, className='flex-container')#https://www.w3schools.com/csS/css3_flexbox_items.asp

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
#debug
#an example of a dataframe...
import pandas as pd
from dash import dash_table

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')

blahhh = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in df.columns],
    data=df.to_dict('records'),
)